In [1]:
import os
import jams
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from joblib import Parallel, delayed
import mir_eval
import pumpp

In [2]:
def getid(fname):
    return os.path.splitext(os.path.basename(fname))[0]

In [3]:
def chord_to_hier(ann):
        
    C3 = pumpp.task.ChordTagTransformer(vocab='3')
    C35 = pumpp.task.ChordTagTransformer(vocab='35')
    C3567s = pumpp.task.ChordTagTransformer(vocab='3567s')
    
    ann_h = jams.Annotation('multi_segment',
                            time=ann.time,
                            duration=ann.duration,
                            annotation_metadata=ann.annotation_metadata)
    
    for obs in ann:
        root = C3.simplify(obs.value)
        if ':' in root:
            root = root[:root.index(':')]
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=1, label=root),
                    confidence=obs.confidence)
        
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=2, label=C3.simplify(obs.value)),
                    confidence=obs.confidence)
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=3, label=C35.simplify(obs.value)),
                    confidence=obs.confidence)
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=4, label=C3567s.simplify(obs.value)),
                    confidence=obs.confidence)
        ann_h.append(time=obs.time, duration=obs.duration,
                    value=dict(level=5, label=obs.value),
                    confidence=obs.confidence)
    return ann_h

In [4]:
def compare(ref_file, est_file, corpus):
    
    ref_jam = jams.load(ref_file, validate=False)
    est_jam = jams.load(est_file, validate=False)
    
    ref_ann = ref_jam.search(corpus=corpus)[0]
    est_ann = est_jam.annotations[0]
    est_ann = est_ann.trim(0, ref_ann.data[-1].time + ref_ann.data[-1].duration)
    
    results = jams.eval.hierarchy(chord_to_hier(ref_ann), chord_to_hier(est_ann))
    
    return {getid(est_file): dict(results)}

In [10]:
from joblib import Parallel, delayed

In [15]:
def score_track(tid, corpus, path):
    try:
        ref_file = os.path.join(REF_PATH, os.path.extsep.join([tid, 'jams']))
        est_file = os.path.join(path, os.path.extsep.join([tid, 'jams']))
        return compare(ref_file, est_file, corpus)
    except ValueError as exc:
        print('Failure at track {}'.format(est_file))
        print(exc)
    return dict()

def eval_model(ids, corpus, path, n_jobs=4):
    
    results_arr = Parallel(n_jobs=n_jobs)(delayed(score_track)(tid, corpus, path) for tid in tqdm(ids))
    results = {}
    for r in results_arr:
        results.update(r)
        
    return pd.DataFrame.from_dict(results, orient='index').sort_index(axis=1)

In [16]:
REF_PATH = '/home/bmcfee/data/eric_chords/references_v2/'

In [17]:
# MARL-mir analysis
marl_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/marl-chords.csv', squeeze=True, header=None)
marl_corp = 'MARL-Chords'

In [18]:
marl_cr2si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "
/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "
/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "
/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


KeyboardInterrupt: 

In [86]:
marl_cr2s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [52]:
marl_cr2 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

A Jupyter Widget

In [53]:
marl_cr1si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

A Jupyter Widget

In [57]:
marl_cr1s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

A Jupyter Widget

In [54]:
marl_cr1 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

A Jupyter Widget

In [7]:
marl_chordino = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

A Jupyter Widget

In [66]:
marl_dnn = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

A Jupyter Widget

In [92]:
marl_khmm = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [93]:
marl_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_chordino_hier.json')
marl_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_dnn_hier.json')
marl_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_khmm_hier.json')

marl_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1_hier.json')
marl_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1s_hier.json')
marl_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1si_hier.json')

marl_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2_hier.json')
marl_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2s_hier.json')
marl_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2si_hier.json')


In [9]:
# Isophonics
iso_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/isophonics.csv', squeeze=True, header=None)
iso_corp = 'Isophonics'

In [10]:
iso_chordino = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

A Jupyter Widget

In [97]:
iso_cr2si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

iso_cr2s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

iso_cr2 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

iso_cr1si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

iso_cr1s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

iso_cr1 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

iso_chordino = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

iso_dnn = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

iso_khmm = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [98]:
iso_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_chordino_hier.json')
iso_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_dnn_hier.json')
iso_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_khmm_hier.json')

iso_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1_hier.json')
iso_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1s_hier.json')
iso_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1si_hier.json')

iso_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2_hier.json')
iso_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2s_hier.json')
iso_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2si_hier.json')


In [ ]:
# Billboard analysis
bb_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/billboard.csv', squeeze=True, header=None)
bb_corp = 'Billboard-Chords'

In [101]:
bb_cr2si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

bb_cr2s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

bb_cr2 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

bb_cr1si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

bb_cr1s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

bb_cr1 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

bb_chordino = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

bb_dnn = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

bb_khmm = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [102]:
bb_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_chordino_hier.json')
bb_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_dnn_hier.json')
bb_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_khmm_hier.json')

bb_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1_hier.json')
bb_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1s_hier.json')
bb_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1si_hier.json')

bb_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2_hier.json')
bb_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2s_hier.json')
bb_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2si_hier.json')
